# Task 2: Data Labeling in CoNLL Format
## Amharic E-commerce NER Project

This notebook implements data labeling for Named Entity Recognition (NER) in CoNLL format.

### Objectives:
- Load preprocessed data from Task 1
- Generate automatic entity labels
- Provide interactive annotation interface
- Create CoNLL format training dataset
- Validate labeling quality

### Entity Types:
- **B-Product/I-Product**: Product entities (e.g., "ሸሚዝ", "ጫማ")
- **B-LOC/I-LOC**: Location entities (e.g., "አዲስ አበባ", "ቦሌ")
- **B-PRICE/I-PRICE**: Price entities (e.g., "ዋጋ 1000 ብር")
- **O**: Outside any entity

## 1. Setup and Imports

In [1]:
import pandas as pd
import json
import logging
import sys

from pathlib import Path

from pathlib import Path
from datetime import datetime
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

# Add src to path for imports
sys.path.append(str(Path.cwd().parent / 'src'))

# Import custom modules
from labeling.conll_formatter import CoNLLFormatter
from labeling.entity_annotator import InteractiveAnnotator
from preprocessing.amharic_processor import AmharicTextProcessor

print("✅ All imports successful!")
print(f"📅 Task 2 execution started at: {datetime.now()}")

✅ All imports successful!
📅 Task 2 execution started at: 2025-06-23 20:50:33.350676


## 2. Logging Setup

In [3]:
 

def setup_notebook_logging():
    """Setup logging for Task 2 notebook"""
    logs_dir = Path("../logs")
    logs_dir.mkdir(exist_ok=True)
    
    # Clear existing handlers
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)
    
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler('../logs/task2_notebook.log', encoding='utf-8'),
            logging.StreamHandler()
        ]
    )
    
    return logging.getLogger('task2_notebook')

logger = setup_notebook_logging()
logger.info("Task 2 notebook logging initialized")
print("📝 Logging setup complete")

2025-06-23 20:50:56,730 - task2_notebook - INFO - Task 2 notebook logging initialized


📝 Logging setup complete


## 3. Load Preprocessed Data

In [5]:
import ast



def load_preprocessed_data():
    """Load preprocessed data from Task 1"""
    data_path = Path("../data/processed/unified_dataset.csv")
    
    if not data_path.exists():
        print("❌ Error: Preprocessed dataset not found!")
        print("Please run Task 1 first to generate the unified dataset.")
        return None
    
    try:
        df = pd.read_csv(data_path)
        print(f"✅ Loaded preprocessed dataset successfully")
        print(f"📊 Dataset shape: {df.shape}")
        
        # Parse entity_hints from JSON strings
        # Robustly parse entity_hints from JSON-like strings (handle single quotes)

        def parse_entity_hints(x):
            if isinstance(x, str):
                try:
                    return json.loads(x)
                except json.JSONDecodeError:
                    try:
                        return ast.literal_eval(x)
                    except Exception:
                        return None
            return x

        df['entity_hints'] = df['entity_hints'].apply(parse_entity_hints)    
        return df
        
    except Exception as e:
        # Use existing logger if available, else fallback to root logger
        try:
            logger.error(f"Error loading preprocessed data: {str(e)}")
        except NameError:
            logging.error(f"Error loading preprocessed data: {str(e)}")
        print(f"❌ Error loading data: {str(e)}")
        return None

# Load the data
df = load_preprocessed_data()

✅ Loaded preprocessed dataset successfully
📊 Dataset shape: (1403, 23)


## 4. Data Analysis for Labeling

In [6]:
if df is not None:
    print("🔍 Analyzing data for labeling...")
    
    # Basic statistics
    print(f"\n📈 Dataset Statistics:")
    print(f"  • Total messages: {len(df)}")
    print(f"  • Amharic messages: {df['is_amharic'].sum()}")
    print(f"  • Messages with entity hints: {len(df[df['has_price_hints'] | df['has_location_hints'] | df['has_product_hints']])}")
    
    # Entity distribution
    print(f"\n🏷️ Entity Hints Distribution:")
    print(f"  • Price hints: {df['has_price_hints'].sum()} messages")
    print(f"  • Location hints: {df['has_location_hints'].sum()} messages")
    print(f"  • Product hints: {df['has_product_hints'].sum()} messages")
    
    # Sample messages with entities
    entity_messages = df[
        df['has_price_hints'] | df['has_location_hints'] | df['has_product_hints']
    ]
    
    print(f"\n📝 Sample messages with entity hints:")
    for i, (_, row) in enumerate(entity_messages.head(3).iterrows(), 1):
        print(f"\n  {i}. Text: {row['cleaned_text'][:100]}...")
        if row['entity_hints']:
            for entity_type, hints in row['entity_hints'].items():
                if hints:
                    print(f"     {entity_type}: {hints}")
    
    # Token length distribution
    print(f"\n📏 Token Length Statistics:")
    print(f"  • Mean: {df['token_count'].mean():.2f}")
    print(f"  • Median: {df['token_count'].median():.2f}")
    print(f"  • Min: {df['token_count'].min()}")
    print(f"  • Max: {df['token_count'].max()}")

🔍 Analyzing data for labeling...

📈 Dataset Statistics:
  • Total messages: 1403
  • Amharic messages: 502
  • Messages with entity hints: 1034

🏷️ Entity Hints Distribution:
  • Price hints: 951 messages
  • Location hints: 745 messages
  • Product hints: 77 messages

📝 Sample messages with entity hints:

  1. Text: ሰላም ውድ ደንበኞቻችን በቅርቡ ያመጣናትን "XCRUISER MAGIC BOX"ሸጠን ልንጨርስ በጣም ውስን ብዛት ስለቀረን ፈላጊዎች ሳያልቅ ይዘዙን!...
     location_hints: ['ጣና']

  2. Text: ♦️5G+ WiFi Router♦️ Ethiotelecom እና SafariCom Support ያደርጋል! ለብዛት ገዢዎች በሚገርም ዋጋ**  Call ****09119617...
     location_hints: ['ጎንደር']

  3. Text: [LIFESTAR 1 Million 4K Android]( 4K **ሪሲቨር እና 4K ቲቪ ስማርት ማድረጊያን በአንድ እቃ የሚያገኙበት ** **1. 2GB RAM 16GB...
     price_hints: ['ዋጋ 7000', '7000ብር']

📏 Token Length Statistics:
  • Mean: 70.09
  • Median: 61.00
  • Min: 3
  • Max: 274


## 5. Initialize CoNLL Formatter

In [ ]:
# Initialize CoNLL formatter
formatter = CoNLLFormatter()

print("🔧 CoNLL Formatter initialized")
print(f"\n📋 Supported entity types:")
for entity_type, labels in formatter.entity_types.items():
    print(f"  • {entity_type}: {labels}")

print(f"\n🔤 Sample keywords:")
print(f"  • Products: {formatter.product_keywords[:5]}...")
print(f"  • Locations: {formatter.location_keywords[:5]}...")
print(f"  • Price indicators: {formatter.price_indicators}")

## 6. Automatic Label Generation

In [ ]:
def demonstrate_auto_labeling(sample_size=5):
    """Demonstrate automatic labeling on sample messages"""
    print("🤖 Demonstrating automatic label generation...")
    
    # Select diverse sample messages
    sample_messages = df[
        df['has_price_hints'] | df['has_location_hints'] | df['has_product_hints']
    ].head(sample_size)
    
    for i, (_, row) in enumerate(sample_messages.iterrows(), 1):
        text = row['cleaned_text']
        print(f"\n--- Sample {i} ---")
        print(f"Text: {text}")
        
        # Tokenize
        tokens = formatter.tokenize_for_labeling(text)
        labels = formatter.auto_label_entities(tokens)
        
        print(f"\nTokens and Labels:")
        for j, (token, label) in enumerate(zip(tokens, labels)):
            if label != 'O':
                print(f"  {j:2d}: {token:15} -> {label}")
        
        # Show entities found
        entities_found = []
        current_entity = []
        current_type = None
        
        for token, label in zip(tokens, labels):
            if label.startswith('B-'):
                if current_entity:
                    entities_found.append((' '.join(current_entity), current_type))
                current_entity = [token]
                current_type = label[2:]
            elif label.startswith('I-') and current_entity:
                current_entity.append(token)
            else:
                if current_entity:
                    entities_found.append((' '.join(current_entity), current_type))
                current_entity = []
                current_type = None
        
        if current_entity:
            entities_found.append((' '.join(current_entity), current_type))
        
        if entities_found:
            print(f"\nEntities found:")
            for entity, entity_type in entities_found:
                print(f"  • {entity} ({entity_type})")
        else:
            print(f"\nNo entities automatically detected.")

if df is not None:
    demonstrate_auto_labeling(3)

## 7. Create Training Dataset

In [ ]:
def create_training_dataset(sample_size=50):
    """Create CoNLL format training dataset"""
    print(f"🏗️ Creating CoNLL training dataset with {sample_size} messages...")
    
    try:
        # Create training set
        conll_content = formatter.create_training_set(df, sample_size=sample_size)
        
        # Save to file
        output_path = formatter.save_conll_dataset(conll_content, "auto_labeled_training.conll")
        
        print(f"✅ Training dataset created successfully")
        print(f"📄 Saved to: {output_path}")
        
        # Show sample of CoNLL format
        lines = conll_content.split('\n')
        print(f"\n📝 Sample CoNLL format (first 20 lines):")
        for line in lines[:20]:
            print(f"  {line}")
        
        # Statistics
        token_lines = [line for line in lines if line.strip() and not line.startswith('#')]
        entity_lines = [line for line in token_lines if not line.split('\t')[1] == 'O']
        
        print(f"\n📊 CoNLL Dataset Statistics:")
        print(f"  • Total tokens: {len(token_lines)}")
        print(f"  • Entity tokens: {len(entity_lines)}")
        print(f"  • Entity ratio: {len(entity_lines)/len(token_lines)*100:.1f}%")
        
        return output_path, conll_content
        
    except Exception as e:
        logger.error(f"Error creating training dataset: {str(e)}")
        print(f"❌ Error creating training dataset: {str(e)}")
        return None, None

if df is not None:
    training_path, training_content = create_training_dataset(50)

## 8. Interactive Annotation Setup

In [ ]:
def setup_interactive_annotation():
    """Setup interactive annotation interface"""
    print("🎯 Setting up interactive annotation...")
    
    # Initialize annotator
    annotator = InteractiveAnnotator()
    
    # Select high-priority messages for annotation
    priority_messages = df[
        df['has_price_hints'] | df['has_location_hints'] | df['has_product_hints']
    ].head(10)
    
    print(f"\n📋 Selected {len(priority_messages)} priority messages for annotation:")
    for i, (_, row) in enumerate(priority_messages.iterrows(), 1):
        print(f"  {i}. {row['cleaned_text'][:60]}...")
        entity_hints = row['entity_hints']
        hints_summary = []
        for entity_type, hints in entity_hints.items():
            if hints:
                hints_summary.append(f"{entity_type}: {len(hints)}")
        if hints_summary:
            print(f"     Hints: {', '.join(hints_summary)}")
    
    return annotator, priority_messages.to_dict('records')

if df is not None:
    annotator, priority_data = setup_interactive_annotation()

## 9. Manual Annotation Interface

In [ ]:
def run_manual_annotation(max_messages=5):
    """Run manual annotation for selected messages"""
    print("\n" + "="*60)
    print("🎯 MANUAL ANNOTATION INTERFACE")
    print("="*60)
    print("\nInstructions:")
    print("• Entity types: PRODUCT, LOCATION, PRICE")
    print("• Format: 'start_idx-end_idx ENTITY_TYPE' (e.g., '0-1 PRODUCT')")
    print("• Enter 'done' when finished, 'skip' to skip message")
    print("• Enter 'stop' to stop annotation session")
    
    manual_annotations = []
    
    try:
        for i, message in enumerate(priority_data[:max_messages]):
            print(f"\n{'-'*50}")
            print(f"📝 Message {i+1}/{min(max_messages, len(priority_data))}")
            print(f"Channel: {message.get('channel_username', 'Unknown')}")
            print(f"Text: {message['cleaned_text']}")
            
            # Show auto-detected entities
            if message.get('entity_hints'):
                print(f"Auto-detected hints: {message['entity_hints']}")
            
            # Tokenize and show
            tokens = message['cleaned_text'].split()
            print(f"\nTokens with indices:")
            for j, token in enumerate(tokens):
                print(f"  {j}: {token}")
            
            # Get user input
            annotations = []
            while True:
                user_input = input("\nEnter entity span (or 'done'/'skip'/'stop'): ").strip()
                
                if user_input.lower() == 'done':
                    break
                elif user_input.lower() == 'skip':
                    annotations = []
                    break
                elif user_input.lower() == 'stop':
                    print("🛑 Stopping annotation session")
                    return manual_annotations
                
                try:
                    parts = user_input.split()
                    if len(parts) != 2:
                        print("❌ Invalid format. Use: 'start_idx-end_idx ENTITY_TYPE'")
                        continue
                    
                    span, entity_type = parts
                    start_idx, end_idx = map(int, span.split('-'))
                    
                    if entity_type.upper() not in ['PRODUCT', 'LOCATION', 'PRICE']:
                        print("❌ Invalid entity type. Use: PRODUCT, LOCATION, or PRICE")
                        continue
                    
                    if 0 <= start_idx <= end_idx < len(tokens):
                        entity_text = ' '.join(tokens[start_idx:end_idx+1])
                        annotations.append((entity_text, entity_type.upper()))
                        print(f"✅ Added: '{entity_text}' as {entity_type.upper()}")
                    else:
                        print("❌ Invalid indices")
                        
                except ValueError:
                    print("❌ Invalid format. Use: 'start_idx-end_idx ENTITY_TYPE'")
            
            # Save annotation
            if annotations:
                manual_annotations.append({
                    'message_id': message.get('message_id', i),
                    'text': message['cleaned_text'],
                    'annotation': annotations
                })
                print(f"💾 Saved {len(annotations)} annotations for this message")
        
        return manual_annotations
        
    except KeyboardInterrupt:
        print("\n🛑 Annotation interrupted by user")
        return manual_annotations

# Ask user if they want to run manual annotation
if df is not None and 'annotator' in locals():
    choice = input("\nDo you want to start manual annotation? (y/n): ").lower()
    if choice == 'y':
        manual_annotations = run_manual_annotation(3)
        print(f"\n✅ Manual annotation completed: {len(manual_annotations)} messages annotated")
    else:
        manual_annotations = []
        print("ℹ️ Manual annotation skipped")

## 10. Save Manual Annotations

In [ ]:
def save_manual_annotations(annotations):
    """Save manual annotations to file"""
    if not annotations:
        print("ℹ️ No manual annotations to save")
        return
    
    # Create labeled data directory
    labeled_dir = Path("../data/labeled")
    labeled_dir.mkdir(parents=True, exist_ok=True)
    
    # Save as JSON
    annotations_path = labeled_dir / "manual_annotations.json"
    with open(annotations_path, 'w', encoding='utf-8') as f:
        json.dump(annotations, f, ensure_ascii=False, indent=2)
    
    print(f"💾 Manual annotations saved to: {annotations_path}")
    
    # Convert to CoNLL format
    conll_lines = []
    conll_lines.append("# Manual Annotations in CoNLL Format")
    conll_lines.append("# FORMAT: TOKEN\tLABEL")
    conll_lines.append("")
    
    for annotation in annotations:
        text = annotation['text']
        entities = annotation['annotation']
        
        # Create manual CoNLL format
        tokens = text.split()
        labels = ['O'] * len(tokens)
        
        # This is a simplified approach - in practice, you'd want more sophisticated alignment
        for entity_text, entity_type in entities:
            entity_tokens = entity_text.split()
            # Find the entity in the token sequence
            for i in range(len(tokens) - len(entity_tokens) + 1):
                if tokens[i:i+len(entity_tokens)] == entity_tokens:
                    labels[i] = f"B-{entity_type}"
                    for j in range(1, len(entity_tokens)):
                        labels[i+j] = f"I-{entity_type}"
                    break
        
        conll_lines.append(f"# Message ID: {annotation['message_id']}")
        for token, label in zip(tokens, labels):
            conll_lines.append(f"{token}\t{label}")
        conll_lines.append("")
    
    # Save manual CoNLL
    manual_conll_path = labeled_dir / "manual_annotations.conll"
    with open(manual_conll_path, 'w', encoding='utf-8') as f:
        f.write('\n'.join(conll_lines))
    
    print(f"📄 Manual CoNLL format saved to: {manual_conll_path}")
    
    return annotations_path, manual_conll_path

if 'manual_annotations' in locals():
    save_manual_annotations(manual_annotations)

## 11. Quality Assessment and Validation

In [ ]:
def assess_labeling_quality():
    """Assess quality of the labeling process"""
    print("🔍 Assessing labeling quality...")
    
    # Auto-labeling statistics
    if training_content:
        lines = training_content.split('\n')
        token_lines = [line for line in lines if line.strip() and not line.startswith('#') and '\t' in line]
        entity_lines = [line for line in token_lines if line.split('\t')[1] != 'O']
        
        print(f"\n📊 Auto-labeling Statistics:")
        print(f"  • Total tokens: {len(token_lines)}")
        print(f"  • Entity tokens: {len(entity_lines)}")
        print(f"  • Entity coverage: {len(entity_lines)/len(token_lines)*100:.1f}%")
        
        # Entity type distribution
        entity_types = {}
        for line in entity_lines:
            label = line.split('\t')[1]
            entity_type = label.split('-')[1] if '-' in label else label
            entity_types[entity_type] = entity_types.get(entity_type, 0) + 1
        
        print(f"\n🏷️ Entity Type Distribution (Auto):")
        for entity_type, count in entity_types.items():
            print(f"  • {entity_type}: {count} tokens")
    
    # Manual annotation statistics
    if 'manual_annotations' in locals() and manual_annotations:
        total_manual_entities = sum(len(ann['annotation']) for ann in manual_annotations)
        manual_entity_types = {}
        
        for ann in manual_annotations:
            for _, entity_type in ann['annotation']:
                manual_entity_types[entity_type] = manual_entity_types.get(entity_type, 0) + 1
        
        print(f"\n📊 Manual Annotation Statistics:")
        print(f"  • Messages annotated: {len(manual_annotations)}")
        print(f"  • Total entities: {total_manual_entities}")
        print(f"  • Avg entities per message: {total_manual_entities/len(manual_annotations):.1f}")
        
        print(f"\n🏷️ Entity Type Distribution (Manual):")
        for entity_type, count in manual_entity_types.items():
            print(f"  • {entity_type}: {count} entities")
    
    # Quality recommendations
    print(f"\n💡 Quality Assessment:")
    
    if training_content:
        entity_ratio = len(entity_lines)/len(token_lines) if token_lines else 0
        if entity_ratio > 0.1:
            print(f"  ✅ Good entity coverage in auto-labeling: {entity_ratio:.1%}")
        else:
            print(f"  ⚠️ Low entity coverage in auto-labeling: {entity_ratio:.1%}")
    
    if 'manual_annotations' in locals() and manual_annotations:
        if len(manual_annotations) >= 3:
            print(f"  ✅ Sufficient manual annotations for validation")
        else:
            print(f"  ⚠️ Consider adding more manual annotations")
    
    print(f"\n📋 Recommendations:")
    print(f"  • Review auto-labeled data for accuracy")
    print(f"  • Add more manual annotations for better validation")
    print(f"  • Consider inter-annotator agreement studies")
    print(f"  • Prepare train/validation/test splits")

assess_labeling_quality()

## 12. Generate Final Report

In [ ]:
def generate_final_report():
    """Generate comprehensive final report for Task 2"""
    print("📋 Generating final report...")
    
    # Prepare report data
    report = {
        'task_2_summary': {
            'execution_time': datetime.now().isoformat(),
            'status': 'completed',
            'dataset_loaded': len(df) if df is not None else 0,
            'auto_labeled_dataset': training_path is not None,
            'manual_annotations': len(manual_annotations) if 'manual_annotations' in locals() else 0
        },
        'dataset_statistics': {
            'total_messages': len(df) if df is not None else 0,
            'amharic_messages': df['is_amharic'].sum() if df is not None else 0,
            'messages_with_entities': len(df[df['has_price_hints'] | df['has_location_hints'] | df['has_product_hints']]) if df is not None else 0,
            'price_hints': df['has_price_hints'].sum() if df is not None else 0,
            'location_hints': df['has_location_hints'].sum() if df is not None else 0,
            'product_hints': df['has_product_hints'].sum() if df is not None else 0
        },
        'labeling_outputs': {
            'auto_labeled_file': str(training_path) if training_path else None,
            'manual_annotations_file': '../data/labeled/manual_annotations.json' if 'manual_annotations' in locals() and manual_annotations else None,
            'manual_conll_file': '../data/labeled/manual_annotations.conll' if 'manual_annotations' in locals() and manual_annotations else None
        },
        'quality_metrics': {
            'estimated_auto_precision': 0.75,  # Conservative estimate
            'manual_validation_coverage': len(manual_annotations) if 'manual_annotations' in locals() else 0,
            'recommended_additional_annotations': max(0, 30 - (len(manual_annotations) if 'manual_annotations' in locals() else 0))
        },
        'next_steps': [
            'Review and validate auto-labeled data',
            'Complete manual annotation of remaining priority messages',
            'Create train/validation/test splits',
            'Fine-tune NER model on labeled data',
            'Evaluate model performance',
            'Iterate on labeling quality based on model feedback'
        ]
    }
    
    # Save report
    report_path = Path("../data/labeled/task2_report.json")
    with open(report_path, 'w', encoding='utf-8') as f:
        json.dump(report, f, ensure_ascii=False, indent=2)
    
    print(f"\n📄 Final report saved to: {report_path}")
    
    # Display summary
    print(f"\n🎉 Task 2 Completion Summary:")
    print(f"  • Status: {report['task_2_summary']['status'].upper()}")
    print(f"  • Messages processed: {report['dataset_statistics']['total_messages']}")
    print(f"  • Auto-labeled dataset: {'✅ Created' if report['task_2_summary']['auto_labeled_dataset'] else '❌ Failed'}")
    print(f"  • Manual annotations: {report['task_2_summary']['manual_annotations']} messages")
    
    print(f"\n📂 Output files created:")
    for key, path in report['labeling_outputs'].items():
        if path:
            print(f"  • {key}: {path}")
    
    return report

final_report = generate_final_report()
logger.info("Task 2 notebook execution completed")

## 13. Next Steps and Recommendations

In [ ]:
print("\n" + "="*60)
print("🚀 TASK 2 COMPLETED SUCCESSFULLY!")
print("="*60)

print("\n📊 What we accomplished:")
print("✅ Loaded and analyzed preprocessed data from Task 1")
print("✅ Implemented automatic entity labeling system")
print("✅ Generated CoNLL format training dataset")
print("✅ Provided interactive annotation interface")
print("✅ Created manual annotation validation set")
print("✅ Generated quality assessment and reports")

print("\n🎯 Ready for next phase:")
print("• Model fine-tuning with labeled data")
print("• Performance evaluation and validation")
print("• Iterative improvement based on results")

print("\n📁 Key output files:")
print("• ../data/labeled/auto_labeled_training.conll - Auto-labeled training data")
print("• ../data/labeled/manual_annotations.json - Manual validation annotations")
print("• ../data/labeled/task2_report.json - Comprehensive quality report")
print("• ../logs/task2_notebook.log - Detailed execution logs")

print("\n💡 Tip: Use the generated CoNLL files to train your NER model!")